# Configuration

In [1]:
import os
from tqdm.notebook import tqdm
from tqdm import tqdm
import librosa
import pandas as pd 
import pandas as np
from sklearn.metrics import classification_report

MODELS_FOLDER = "models"
SCALERS_FOLDER = "scalers"

# METTERE PATH MODELS E SCLAERS E CALCOLARE LA FINAL PREDICTION DIVERSAMENTE PRENDENDENDO PEZZI DI CODICE DELL'ENSEMBLE

In [2]:
# import utils
import feature_extraction as fe
import emotion_predictions as ep
import scipy
import numpy as np
from scipy import signal
from scipy.io.wavfile import write
#from utils import resample, denoise
import sklearn
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# print('The scikit-learn version is {}.'.format(sklearn.__version__))

# Clean Data - Compute dataframes for datasets and split in Train, Val, Test 

In [3]:
main_path = '/Users/helemanc/Documents/MasterAI/THESIS/Datasets SER'
TESS = os.path.join(main_path, "tess/TESS Toronto emotional speech set data/") 
RAV = os.path.join(main_path, "ravdess-emotional-speech-audio/audio_speech_actors_01-24")
SAVEE = os.path.join(main_path, "savee/ALL/")
CREMA = os.path.join(main_path, "creamd/AudioWAV/")

## RAVDESS

In [4]:
lst = []
emotion = []
voc_channel = []
full_path = []
modality = []
intensity = []
actors = []
phrase =[]

for root, dirs, files in tqdm(os.walk(RAV)):
    for file in files:
        try:
            #Load librosa array, obtain mfcss, store the file and the mfcss information in a new array
            # X, sample_rate = librosa.load(os.path.join(root,file), res_type='kaiser_fast')
            # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
            # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
            # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
           
            modal = int(file[1:2])
            vchan = int(file[4:5])
            lab = int(file[7:8])
            ints = int(file[10:11])
            phr = int(file[13:14])
            act = int(file[18:20])
            # arr = mfccs, lab
            # lst.append(arr)
            
            modality.append(modal)
            voc_channel.append(vchan)
            emotion.append(lab) #only labels
            intensity.append(ints)
            phrase.append(phr)
            actors.append(act)
            
            full_path.append((root, file)) # only files
          # If the file is not valid, skip it
        except ValueError:
            continue

25it [00:00, 316.37it/s]


In [5]:
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# merge neutral and calm
emotions_list = ['neutral', 'neutral', 'happy', 'sadness', 'angry', 'fear', 'disgust', 'surprise']
emotion_dict = {em[0]+1:em[1] for em in enumerate(emotions_list)}

df = pd.DataFrame([emotion, voc_channel, modality, intensity, actors, actors,phrase, full_path]).T
df.columns = ['emotion', 'voc_channel', 'modality', 'intensity', 'actors', 'gender', 'phrase', 'path']
df['emotion'] = df['emotion'].map(emotion_dict)
df['voc_channel'] = df['voc_channel'].map({1: 'speech', 2:'song'})
df['modality'] = df['modality'].map({1: 'full AV', 2:'video only', 3:'audio only'})
df['intensity'] = df['intensity'].map({1: 'normal', 2:'strong'})
df['actors'] = df['actors']
df['gender'] = df['actors'].apply(lambda x: 'female' if x%2 == 0 else 'male')
df['phrase'] = df['phrase'].map({1: 'Kids are talking by the door', 2:'Dogs are sitting by the door'})
df['path'] = df['path'].apply(lambda x: x[0] + '/' + x[1])

In [6]:
# remove files with noise to apply the same noise to all files for data augmentation 
df = df[~df.path.str.contains('noise')]

In [7]:
df.head()

,emotion,voc_channel,modality,intensity,actors,gender,phrase,path
0,angry,speech,audio only,normal,16,female,Dogs are sitting by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,fear,speech,audio only,normal,16,female,Dogs are sitting by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,fear,speech,audio only,strong,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,angry,speech,audio only,strong,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...
5,disgust,speech,audio only,normal,16,female,Kids are talking by the door,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [8]:
# only speech
RAV_df = df
RAV_df = RAV_df.loc[RAV_df.voc_channel == 'speech']

In [9]:
RAV_df.insert(0, "emotion_label", RAV_df.emotion, True)

In [10]:
RAV_df = RAV_df.drop(['emotion', 'voc_channel', 'modality', 'intensity', 'phrase'], 1)

In [11]:
RAV_df

,emotion_label,actors,gender,path
0,angry,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,fear,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,fear,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,angry,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
5,disgust,16,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
...,...,...,...,...
2869,happy,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2871,happy,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2872,neutral,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2873,neutral,8,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [12]:
RAV_train = []
RAV_val = []
RAV_test = []

In [13]:
for index, row in RAV_df.iterrows():
    if row['actors'] in range(1,21): 
        RAV_train.append(row) 
    elif row['actors'] in range(21,23): 
        RAV_val.append(row)
    elif row['actors'] in range(23,25): 
        RAV_test.append(row)
len(RAV_train), len(RAV_val), len(RAV_test)

(1200, 120, 120)

In [14]:
RAV_train = pd.DataFrame(RAV_train)
RAV_val = pd.DataFrame(RAV_val)
RAV_test = pd.DataFrame(RAV_test)

In [15]:
RAV_train = RAV_train.drop(['actors'], 1)
RAV_val = RAV_val.drop(['actors'], 1)
RAV_test = RAV_test.drop(['actors'], 1)

In [16]:
RAV_train.reset_index(drop=True, inplace = True) 
RAV_val.reset_index(drop=True, inplace = True) 
RAV_test.reset_index(drop=True, inplace = True ) 

## SAVEE

In [17]:
# Get the data location for SAVEE
dir_list = os.listdir(SAVEE)

# parse the filename to get the emotions
emotion=[]
path = []
actors = []
gender = []
for i in dir_list:
    actors.append(i[:2])
    if i[-8:-6]=='_a':
        emotion.append('angry')
        gender.append('male')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
        gender.append('male')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
        gender.append('male')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
        gender.append('male')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
        gender.append('male')
    elif i[-8:-6]=='sa':
        emotion.append('sadness')
        gender.append('male')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
        gender.append('male') 
    else:
        emotion.append('Unknown') 
    path.append(SAVEE + i)
    
# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['emotion_label'])
                      
SAVEE_df = pd.concat([SAVEE_df,
                      pd.DataFrame(actors, columns = ['actors']),
                      pd.DataFrame(gender, columns = ['gender']), 
                      pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.emotion_label.value_counts()

neutral     120
fear         60
surprise     60
angry        60
sadness      60
disgust      60
happy        60
Name: emotion_label, dtype: int64

In [18]:
SAVEE_df.head()

,emotion_label,actors,gender,path
0,sadness,JK,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,sadness,JK,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,neutral,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,surprise,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,neutral,DC,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [19]:
SAVEE_train = []
SAVEE_val = []
SAVEE_test = []

In [20]:
#DC, JE, JK, KL
for index, row in SAVEE_df.iterrows(): 
    if row['actors'] == 'DC' or row ['actors'] == 'JE':
        SAVEE_train.append(row)
    elif row['actors'] == 'JK': 
        SAVEE_val.append(row)
    else: 
        df_test.append(row)
len(SAVEE_train), len(SAVEE_val), len(SAVEE_test)

(240, 120, 120)

In [21]:
SAVEE_train = pd.DataFrame(SAVEE_train)
SAVEE_val = pd.DataFrame(SAVEE_val)
df_test = pd.DataFrame(SAVEE_test)

In [22]:
SAVEE_train = SAVEE_train.drop(['actors'], 1)
SAVEE_val = SAVEE_val.drop(['actors'], 1)
SAVEE_test = SAVEE_test.drop(['actors'], 1)

In [23]:
SAVEE_train = SAVEE_train.reset_index(drop=True) 
SAVEE_val = SAVEE_val.reset_index(drop=True) 
SAVEE_test = SAVEE_test.reset_index(drop=True) 

## TESS

In [24]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

path = []
emotion = []
gender = []
actors = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry':
            emotion.append('angry')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_angry': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_disgust' :
            emotion.append('disgust')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_disgust': 
            emotion.append('disgust')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_Fear':
            emotion.append('fear')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_fear': 
            emotion.append('fear')
            gender.append('female')
            actors.append('YAF') 
            
            
        elif i == 'OAF_happy' :
            emotion.append('happy')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_happy': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_neutral':
            emotion.append('neutral')
            gender.append('female')
            actors.append('OAF')   
        elif i == 'YAF_neutral': 
            emotion.append('neutral')
            gender.append('female')
            actors.append('YAF')      
            
                
        elif i == 'OAF_Pleasant_surprise':
            emotion.append('surprise')
            gender.append('female')
            actors.append('OAF')
        
        elif i == 'YAF_pleasant_surprised': 
            emotion.append('surprise')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_Sad':
            emotion.append('sadness')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_sad': 
            emotion.append('sadness')
            gender.append('female')
            actors.append('YAF')            
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['emotion_label'])
TESS_df = pd.concat([TESS_df, pd.DataFrame(gender, columns = ['gender']), 
                     pd.DataFrame(actors, columns= ['actors']),
                     pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.emotion_label.value_counts()

angry       1200
neutral      800
disgust      800
surprise     800
fear         800
sadness      800
happy        400
Name: emotion_label, dtype: int64

In [25]:
TESS_df= TESS_df[~TESS_df.path.str.contains('noise')]

In [26]:
TESS_train = []
TESS_test = []

In [27]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [28]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [29]:
TESS_train = TESS_train.drop(['actors'], 1)
TESS_test = TESS_test.drop(['actors'], 1)

In [30]:
TESS_train = TESS_train.reset_index(drop=True) 
TESS_test  = TESS_test.reset_index(drop=True) 

## CREMA-D

In [31]:
males = [1,
5,
11,
14,
15,
16,
17,
19,
22,
23,
26,
27,
31,
32,
33,
34,
35,
36,
38,
39,
41,
42,
44,
45,
48,
50,
51,
57,
59, 
62, 
64,
65, 
66,
67,
68,
69,
70,
71,
77, 
80, 
81, 
83, 
85, 
86, 
87,
88, 
90]

In [32]:
females = [ 2,
3,
4,
6,
7,
8,
9,
10,
12,
13,
18,
20,
21,
24,
25,
28,
29,
30,
37,
40,
43,
46,
47,
49,
52,
53,
54,
55,
56, 
58, 
60,
61,
63,
72, 
73, 
74, 
75, 
76, 
78, 
79, 
82, 
84, 
89, 
91]

In [33]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
actors = []
gender = []




for file in crema_directory_list:

    # storing file emotions
    part=file.split('_')
    
    # use only high intensity files
    if "HI" in part[3] :
        actor = part[0][2:]
        actors.append(actor)
        if int(actor) in males:
            gender.append('male')
        else: 
            gender.append('female')
    
        # storing file paths
        file_path.append(CREMA + file)
        if part[2] == 'SAD':
            file_emotion.append('sadness')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['emotion_label'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
actors_df = pd.DataFrame(actors, columns=['actors'])
gender_df = pd.DataFrame(gender, columns=['gender'])                      
Crema_df = pd.concat([emotion_df, actors_df, gender_df, path_df], axis=1)
Crema_df.head()

,emotion_label,actors,gender,path
0,sadness,28,female,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,angry,48,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,disgust,27,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,disgust,32,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,happy,80,male,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [34]:
Crema_df.shape

(455, 4)

In [35]:
actor_files = {}

for index, row in Crema_df.iterrows():
    actor = row['actors']
    if actor not in actor_files.keys(): 
        actor_files[actor] = 1
    else: 
        actor_files[actor]+=1

In [36]:
actor_files

{'28': 5,
 '48': 5,
 '27': 5,
 '32': 5,
 '80': 5,
 '59': 5,
 '39': 5,
 '43': 5,
 '56': 5,
 '31': 5,
 '24': 5,
 '51': 5,
 '44': 5,
 '77': 5,
 '62': 5,
 '88': 5,
 '55': 5,
 '40': 5,
 '13': 5,
 '06': 5,
 '35': 5,
 '20': 5,
 '87': 5,
 '09': 5,
 '78': 5,
 '65': 5,
 '70': 5,
 '23': 5,
 '36': 5,
 '01': 5,
 '14': 5,
 '47': 5,
 '52': 5,
 '58': 5,
 '37': 5,
 '22': 5,
 '71': 5,
 '64': 5,
 '50': 5,
 '45': 5,
 '57': 5,
 '42': 5,
 '29': 5,
 '53': 5,
 '46': 5,
 '33': 5,
 '26': 5,
 '34': 5,
 '21': 5,
 '15': 5,
 '08': 5,
 '86': 5,
 '79': 5,
 '63': 5,
 '76': 5,
 '25': 5,
 '30': 5,
 '07': 5,
 '12': 5,
 '41': 5,
 '54': 5,
 '89': 5,
 '81': 5,
 '49': 5,
 '38': 5,
 '05': 5,
 '10': 5,
 '61': 5,
 '74': 5,
 '69': 5,
 '18': 5,
 '83': 5,
 '17': 5,
 '02': 5,
 '73': 5,
 '66': 5,
 '84': 5,
 '91': 5,
 '90': 5,
 '85': 5,
 '03': 5,
 '16': 5,
 '67': 5,
 '72': 5,
 '68': 5,
 '82': 5,
 '19': 5,
 '11': 5,
 '04': 5,
 '75': 5,
 '60': 5}

In [37]:
count_males = 0 
count_females = 0 
male_list = []
for index, row in Crema_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1

In [38]:
count_males, count_females

(235, 220)

Since there are more males than females we will remove randomly 3 male actors (since there are exactly 5 audio files per actor)

In [39]:
import random 
'''
random.seed(42)
males_to_remove = random.sample(male_list, 3)
males_to_remove
'''
males_to_remove = ['17', '80', '88']

In [40]:
new_df = []
for index, row in Crema_df.iterrows(): 
    if row['actors'] not in males_to_remove: 
        new_df.append(row)

In [41]:
CREMA_df = pd.DataFrame(new_df)

In [42]:
for index, row in CREMA_df.iterrows(): 
    if row['actors'] == '17': 
        print("Elements not removed")

In [43]:
count_males = 0 
count_females = 0 
male_list = []
female_list = []
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1
        if actor not in female_list: 
            female_list.append(actor)

In [44]:
count_males, count_females

(220, 220)

In [45]:
len(female_list)

44

In [46]:
len(male_list)

44

In [47]:
CREMA_train = []
CREMA_val = []
CREMA_test = []

In [48]:
females_train = random.sample(female_list, 32)
males_train = random.sample(male_list, 32)

# remove the elements assigned to train 
for element in females_train:
    if element in female_list:
        female_list.remove(element)
        
for element in males_train:
    if element in male_list:
        male_list.remove(element)

         
females_val = random.sample(female_list, 6) 
males_val = random.sample(male_list, 6) 

# remove the elements assigned to val
for element in females_val:
    if element in female_list:
        female_list.remove(element)
        
for element in males_val:
    if element in male_list:
        male_list.remove(element)
        
females_test = random.sample(female_list, 6) 
males_test = random.sample(male_list, 6)        

In [49]:
females_train, males_train, females_val, males_val, females_test, males_test

(['24',
  '09',
  '84',
  '40',
  '72',
  '29',
  '82',
  '18',
  '28',
  '91',
  '47',
  '73',
  '60',
  '54',
  '13',
  '25',
  '55',
  '78',
  '37',
  '63',
  '52',
  '53',
  '10',
  '75',
  '21',
  '02',
  '46',
  '30',
  '04',
  '06',
  '74',
  '08'],
 ['45',
  '44',
  '22',
  '59',
  '26',
  '14',
  '71',
  '67',
  '69',
  '65',
  '33',
  '11',
  '35',
  '66',
  '57',
  '15',
  '01',
  '87',
  '77',
  '31',
  '90',
  '68',
  '81',
  '23',
  '42',
  '51',
  '50',
  '86',
  '05',
  '70',
  '64',
  '48'],
 ['89', '20', '49', '79', '43', '58'],
 ['16', '41', '36', '32', '39', '83'],
 ['76', '03', '61', '12', '56', '07'],
 ['34', '38', '19', '62', '27', '85'])

In [50]:
train = females_train + males_train 
val = females_val + males_val 
test = females_test + males_test

In [51]:
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if actor in train: 
        CREMA_train.append(row)
    elif actor in val: 
        CREMA_val.append(row)
    else:
        CREMA_test.append(row)

In [52]:
CREMA_train = pd.DataFrame(CREMA_train) 
CREMA_val = pd.DataFrame(CREMA_val) 
CREMA_test = pd.DataFrame(CREMA_test)

In [53]:
CREMA_train.shape, CREMA_val.shape, CREMA_test.shape

((320, 4), (60, 4), (60, 4))

In [54]:
CREMA_train = CREMA_train.drop(['actors'], 1)
CREMA_val = CREMA_val.drop(['actors'], 1)
CREMA_test = CREMA_test.drop(['actors'], 1)

In [55]:
CREMA_train = CREMA_train.reset_index(drop=True) 
CREMA_val = CREMA_val.reset_index(drop = True) 
CREMA_test = CREMA_test.reset_index(drop = True)

# Utils

In [70]:
def denoise(samples):
    """
    :param samples: an array representing the sampled audio file
    :type samples: float
    :return: an array representing the clean audio file
    :rtype: float
    """
    samples_wiener = scipy.signal.wiener(samples)
    return samples_wiener

def resample(input_data, sample_rate, required_sample_rate=16000, amplify=False):
    """
    Resampling function. Takes an audio with sample_rate and resamples
    it to required_sample_rate. Optionally, amplifies volume.
    ​
    :param input_data: Input audio data
    :type input_data: numpy.ndarray
    :param sample_rate: Sample rate of input audio data
    :type sample_rate: int
    :param required_sample_rate: Sample rate to resample original audio, defaults to 16000
    :type required_sample_rate: int, optional
    :param amplify: Whether to amplify audio volume, defaults to False
    :type amplify: bool, optional
    :return: Resampled audio and new sample rate
    :rtype: numpy.ndarray, int
    """
    if sample_rate < required_sample_rate:
        resampling_factor = int(round(required_sample_rate/sample_rate, 0))
        new_rate = sample_rate * resampling_factor
        samples = len(input_data) * resampling_factor
        resampled = signal.resample(input_data, samples)
    elif sample_rate > required_sample_rate:
        resampling_factor = int(round(sample_rate/required_sample_rate, 0))
        new_rate = int(sample_rate / resampling_factor)
        resampled = signal.decimate(input_data, resampling_factor)
    else:
        resampling_factor = 1
        new_rate = sample_rate
        resampled = input_data
    if amplify and input_data.size > 0:
        absolute_values = np.absolute(resampled)
        max_value = np.amax(absolute_values)
        max_range = np.iinfo(np.int16).max
        amplify_factor = max_range/max_value
        resampled = resampled * amplify_factor
        resampled = resampled.round()
    return resampled.astype(np.float32), new_rate


def make_predictions(dataset, labels, model_name): 
    predictions = []
    model_predictions_list = []
    counter = 0
    for filepath in tqdm(dataset['path']):
        samples, sample_rate = fe.read_file(filepath)
        samples, sample_rate = resample(samples, sample_rate)
        new_samples = fe.cut_pad(samples)
        #new_filepath = "tmp.wav"
        for dirpath, dirnames, filenames in os.walk(MODELS_FOLDER):
            # iterate over the list of dirnames to load the convmodel
            # iterate over the list of filenames to get the svm model
            for model in dirnames:
                model_type = 'conv'
                parts = model.split('_')
                num_exp = parts[1]
                num_data = parts[2].split('.')[0]
                id_exp = num_exp + '_' + num_data
                if id_exp == model_name: 
                    model_path = os.path.join(dirpath, model)
                    conv_model = tf.keras.models.load_model(model_path)
                    for scaler in os.listdir(SCALERS_FOLDER):
                        parts = scaler.split('_')
                        num_exp = parts[1]
                        num_data = parts[2].split('.')[0]
                        id_exp_scaler = num_exp + '_' + num_data
                        if id_exp == id_exp_scaler:
                            # print("Loading scaler: ", scaler)
                            scaler_conv_model = scaler
                            mfccs = fe.mfccs_scaled(new_samples, scaler_conv_model, id_exp)
                            pred = ep.make_predictions(conv_model, model_type, mfccs, 'majority')
                            # add result to list and dictionary
                            predictions.append(pred)
                            #model_predictions[id_exp] = pred
            break
        #final_prediction, model_predictions = ensemble.ensemble(new_samples, prediction_scheme, return_model_predictions = True)

        #predictions.append(final_prediction)
        #model_predictions_list.append(model_predictions) 
        print("True label", labels[counter], "Predicted label", predictions[counter])
        counter+=1
    return predictions

In [71]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

# Transfer RAVDESS -> SAVEE

In [159]:
df_test = SAVEE_test

## 1_1

In [63]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_1')

  1%|          | 1/120 [00:00<01:52,  1.06it/s]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:01<01:50,  1.07it/s]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:02<01:50,  1.06it/s]

...

True label 1 Predicted label 1


 99%|█████████▉| 119/120 [02:37<00:01,  1.27s/it]

True label 1 Predicted label 0


100%|██████████| 120/120 [02:39<00:00,  1.33s/it]

True label 1 Predicted label 1


In [72]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [73]:
df_res

,true_label,pred_label,path
0,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
...,...,...,...
115,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
116,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
117,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
118,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [74]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.46      0.30      0.36        60
           1       0.48      0.65      0.55        60

    accuracy                           0.48       120
   macro avg       0.47      0.47      0.46       120
weighted avg       0.47      0.47      0.46       120



## 1_2

In [76]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_2')

  1%|          | 1/120 [00:01<03:32,  1.78s/it]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:02<03:01,  1.54s/it]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:03<02:30,  1.29s/it]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [01:52<00:00,  1.36it/s]

True label 1 Predicted label 0


100%|██████████| 120/120 [01:53<00:00,  1.06it/s]

True label 1 Predicted label 1


In [77]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [78]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [79]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.44      0.40      0.42        60
           1       0.45      0.50      0.48        60

    accuracy                           0.45       120
   macro avg       0.45      0.45      0.45       120
weighted avg       0.45      0.45      0.45       120



## 4_2

In [80]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_2')

  1%|          | 1/120 [00:01<02:09,  1.08s/it]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:01<01:51,  1.06it/s]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:02<01:42,  1.14it/s]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [02:03<00:00,  1.25it/s]

True label 1 Predicted label 1


100%|██████████| 120/120 [02:04<00:00,  1.04s/it]

True label 1 Predicted label 1


In [81]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [82]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [83]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.55      0.30      0.39        60
           1       0.52      0.75      0.61        60

    accuracy                           0.53       120
   macro avg       0.53      0.53      0.50       120
weighted avg       0.53      0.53      0.50       120



## 2_1

In [84]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_1')

  1%|          | 1/120 [00:00<01:27,  1.36it/s]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:01<01:25,  1.39it/s]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:02<01:24,  1.39it/s]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [01:43<00:00,  1.18it/s]

True label 1 Predicted label 0


100%|██████████| 120/120 [01:44<00:00,  1.15it/s]

True label 1 Predicted label 0


In [85]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [86]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [87]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.45      0.48      0.46        60
           1       0.44      0.40      0.42        60

    accuracy                           0.44       120
   macro avg       0.44      0.44      0.44       120
weighted avg       0.44      0.44      0.44       120



## 2_2

In [88]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_2')

  1%|          | 1/120 [00:01<02:11,  1.11s/it]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:01<01:56,  1.01it/s]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:02<01:46,  1.10it/s]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [02:16<00:01,  1.05s/it]

True label 1 Predicted label 0


100%|██████████| 120/120 [02:17<00:00,  1.14s/it]

True label 1 Predicted label 1


In [89]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [90]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [91]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.51      0.60      0.55        60
           1       0.52      0.43      0.47        60

    accuracy                           0.52       120
   macro avg       0.52      0.52      0.51       120
weighted avg       0.52      0.52      0.51       120



## 4_1

In [92]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_1')

  1%|          | 1/120 [00:01<03:18,  1.67s/it]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:03<03:09,  1.61s/it]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:04<03:06,  1.60s/it]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [02:19<00:01,  1.60s/it]

True label 1 Predicted label 0


100%|██████████| 120/120 [02:20<00:00,  1.17s/it]

True label 1 Predicted label 0


In [93]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [94]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [95]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.50      0.48      0.49        60
           1       0.50      0.52      0.51        60

    accuracy                           0.50       120
   macro avg       0.50      0.50      0.50       120
weighted avg       0.50      0.50      0.50       120



## 3_1

In [96]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_1')

  1%|          | 1/120 [00:03<07:00,  3.53s/it]

True label 1 Predicted label 0


  2%|▏         | 2/120 [00:05<05:47,  2.95s/it]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:06<04:50,  2.48s/it]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [02:06<00:00,  1.07it/s]

True label 1 Predicted label 0


100%|██████████| 120/120 [02:07<00:00,  1.06s/it]

True label 1 Predicted label 0


In [97]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [98]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [99]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.52      0.75      0.62        60
           1       0.56      0.32      0.40        60

    accuracy                           0.53       120
   macro avg       0.54      0.53      0.51       120
weighted avg       0.54      0.53      0.51       120



## 3_2

In [160]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_2')

  1%|          | 1/120 [00:03<07:22,  3.72s/it]

True label 1 Predicted label 1


  2%|▏         | 2/120 [00:05<06:08,  3.13s/it]

True label 1 Predicted label 0


  2%|▎         | 3/120 [00:07<05:11,  2.66s/it]

...

True label 1 Predicted label 0


 99%|█████████▉| 119/120 [02:46<00:01,  1.09s/it]

True label 1 Predicted label 0


100%|██████████| 120/120 [02:47<00:00,  1.40s/it]

True label 1 Predicted label 1


In [161]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [162]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [163]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.45      0.25      0.32        60
           1       0.48      0.70      0.57        60

    accuracy                           0.48       120
   macro avg       0.47      0.47      0.45       120
weighted avg       0.47      0.47      0.45       120



# Transfer RAVDESS -> TESS

In [164]:
df_test = TESS_test

## 1_1

In [101]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_1')

  0%|          | 1/1400 [00:00<19:01,  1.23it/s]

True label 1 Predicted label 0


  0%|          | 2/1400 [00:01<17:09,  1.36it/s]

True label 1 Predicted label 1


  0%|          | 3/1400 [00:02<16:30,  1.41it/s]

...

True label 0 Predicted label 1


100%|█████████▉| 1399/1400 [28:27<00:01,  1.20s/it]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [28:28<00:00,  1.22s/it]

True label 0 Predicted label 1


In [102]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [103]:
df_res

,true_label,pred_label,path
0,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
...,...,...,...
1395,0,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1396,0,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1397,0,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1398,0,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [104]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.52      0.74      0.61       600
           1       0.71      0.48      0.58       800

    accuracy                           0.59      1400
   macro avg       0.62      0.61      0.59      1400
weighted avg       0.63      0.59      0.59      1400



## 1_2

In [106]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_2')

  0%|          | 1/1400 [00:05<2:18:05,  5.92s/it]

True label 1 Predicted label 0


  0%|          | 2/1400 [00:14<2:34:44,  6.64s/it]

True label 1 Predicted label 1


  0%|          | 3/1400 [00:16<2:05:37,  5.40s/it]

...

True label 0 Predicted label 0


100%|█████████▉| 1399/1400 [20:01<00:00,  1.63it/s]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [20:02<00:00,  1.16it/s]

True label 0 Predicted label 0


In [107]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [108]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [109]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.50      0.76      0.60       600
           1       0.71      0.43      0.54       800

    accuracy                           0.57      1400
   macro avg       0.60      0.60      0.57      1400
weighted avg       0.62      0.57      0.57      1400



## 4_2

In [110]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_2')

  0%|          | 1/1400 [00:00<12:57,  1.80it/s]

True label 1 Predicted label 0


  0%|          | 2/1400 [00:01<12:48,  1.82it/s]

True label 1 Predicted label 1


  0%|          | 3/1400 [00:02<15:22,  1.51it/s]

...

True label 0 Predicted label 0


100%|█████████▉| 1399/1400 [14:46<00:00,  1.67it/s]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [14:47<00:00,  1.58it/s]

True label 0 Predicted label 0


In [111]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [112]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [113]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.54      0.83      0.65       600
           1       0.79      0.47      0.59       800

    accuracy                           0.62      1400
   macro avg       0.66      0.65      0.62      1400
weighted avg       0.68      0.62      0.61      1400



## 2_1

In [114]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_1')

  0%|          | 1/1400 [00:00<14:27,  1.61it/s]

True label 1 Predicted label 1


  0%|          | 2/1400 [00:01<13:46,  1.69it/s]

True label 1 Predicted label 1


  0%|          | 3/1400 [00:01<13:10,  1.77it/s]

...

True label 0 Predicted label 0


100%|█████████▉| 1399/1400 [14:13<00:00,  1.74it/s]

True label 0 Predicted label 0


100%|██████████| 1400/1400 [14:13<00:00,  1.64it/s]

True label 0 Predicted label 0


In [115]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [116]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [117]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.49      0.91      0.64       600
           1       0.81      0.29      0.43       800

    accuracy                           0.56      1400
   macro avg       0.65      0.60      0.53      1400
weighted avg       0.67      0.56      0.52      1400



## 2_2

In [118]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_2')

  0%|          | 1/1400 [00:00<13:05,  1.78it/s]

True label 1 Predicted label 0


  0%|          | 2/1400 [00:01<12:46,  1.82it/s]

True label 1 Predicted label 0


  0%|          | 3/1400 [00:01<12:35,  1.85it/s]

...

True label 0 Predicted label 0


100%|█████████▉| 1399/1400 [13:59<00:00,  1.60it/s]

True label 0 Predicted label 0


100%|██████████| 1400/1400 [14:00<00:00,  1.67it/s]

True label 0 Predicted label 0


In [119]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [120]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [121]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.46      0.96      0.62       600
           1       0.84      0.15      0.25       800

    accuracy                           0.50      1400
   macro avg       0.65      0.56      0.44      1400
weighted avg       0.68      0.50      0.41      1400



## 4_1

In [122]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_1')

  0%|          | 1/1400 [00:00<12:29,  1.87it/s]

True label 1 Predicted label 1


  0%|          | 2/1400 [00:01<12:58,  1.80it/s]

True label 1 Predicted label 1


  0%|          | 3/1400 [00:01<12:38,  1.84it/s]

...

True label 0 Predicted label 1


100%|█████████▉| 1399/1400 [14:02<00:00,  1.56it/s]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [14:03<00:00,  1.66it/s]

True label 0 Predicted label 0


In [123]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [124]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [125]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.53      0.90      0.66       600
           1       0.84      0.39      0.53       800

    accuracy                           0.61      1400
   macro avg       0.68      0.65      0.60      1400
weighted avg       0.70      0.61      0.59      1400



## 3_1

In [126]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_1')

  0%|          | 1/1400 [00:00<12:56,  1.80it/s]

True label 1 Predicted label 1


  0%|          | 2/1400 [00:01<12:35,  1.85it/s]

True label 1 Predicted label 0


  0%|          | 3/1400 [00:01<12:51,  1.81it/s]

...

True label 0 Predicted label 0


100%|█████████▉| 1399/1400 [14:18<00:00,  1.49it/s]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [14:18<00:00,  1.63it/s]

True label 0 Predicted label 0


In [127]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [128]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [129]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.50      0.86      0.63       600
           1       0.77      0.35      0.48       800

    accuracy                           0.57      1400
   macro avg       0.63      0.60      0.55      1400
weighted avg       0.65      0.57      0.54      1400



## 3_2

In [165]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_2')

  0%|          | 1/1400 [00:01<24:24,  1.05s/it]

True label 1 Predicted label 1


  0%|          | 2/1400 [00:02<23:59,  1.03s/it]

True label 1 Predicted label 0


  0%|          | 3/1400 [00:02<23:19,  1.00s/it]

...

True label 0 Predicted label 1


100%|█████████▉| 1399/1400 [36:34<00:01,  1.09s/it]

True label 0 Predicted label 1


100%|██████████| 1400/1400 [36:35<00:00,  1.57s/it]

True label 0 Predicted label 1


In [166]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [167]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [168]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.61      0.85      0.71       600
           1       0.84      0.59      0.70       800

    accuracy                           0.70      1400
   macro avg       0.73      0.72      0.70      1400
weighted avg       0.74      0.70      0.70      1400



# Transfer RAVDESS -> CREMA

In [169]:
df_test = CREMA_test

## 1_1

In [131]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_1')

  2%|▏         | 1/60 [00:00<00:48,  1.22it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:47,  1.23it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:02<00:45,  1.27it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [00:35<00:00,  1.71it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [00:36<00:00,  1.64it/s]

True label 1 Predicted label 0


In [132]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [133]:
df_res

,true_label,pred_label,path
0,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
1,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
2,0,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
3,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
4,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
5,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
6,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
7,1,0,/Users/helemanc/Documents/MasterAI/THESIS/Data...
8,1,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...
9,0,1,/Users/helemanc/Documents/MasterAI/THESIS/Data...


In [134]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.21      0.83      0.33        12
           1       0.83      0.21      0.33        48

    accuracy                           0.33        60
   macro avg       0.52      0.52      0.33        60
weighted avg       0.71      0.33      0.33        60



## 1_2

In [135]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '1_2')

  2%|▏         | 1/60 [00:00<00:55,  1.07it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:46,  1.24it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:01<00:41,  1.39it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [00:34<00:00,  1.72it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [00:34<00:00,  1.72it/s]

True label 1 Predicted label 0


In [136]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [137]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [138]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.19      0.75      0.31        12
           1       0.77      0.21      0.33        48

    accuracy                           0.32        60
   macro avg       0.48      0.48      0.32        60
weighted avg       0.65      0.32      0.32        60



## 4_2

In [139]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_2')

  2%|▏         | 1/60 [00:00<00:30,  1.90it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:36,  1.59it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:01<00:34,  1.64it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [00:34<00:00,  1.63it/s]

True label 0 Predicted label 1


100%|██████████| 60/60 [00:35<00:00,  1.70it/s]

True label 1 Predicted label 1


In [140]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [141]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [142]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.22      0.58      0.32        12
           1       0.82      0.48      0.61        48

    accuracy                           0.50        60
   macro avg       0.52      0.53      0.46        60
weighted avg       0.70      0.50      0.55        60



## 2_1

In [143]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_1')

  2%|▏         | 1/60 [00:00<00:31,  1.90it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:30,  1.91it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:01<00:29,  1.93it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [00:35<00:00,  1.51it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [00:36<00:00,  1.66it/s]

True label 1 Predicted label 1


In [144]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [145]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [146]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.20      0.75      0.32        12
           1       0.81      0.27      0.41        48

    accuracy                           0.37        60
   macro avg       0.51      0.51      0.36        60
weighted avg       0.69      0.37      0.39        60



## 2_2

In [147]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '2_2')

  2%|▏         | 1/60 [00:00<00:43,  1.36it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:41,  1.39it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:02<00:40,  1.42it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [01:26<00:00,  1.49it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [01:26<00:00,  1.44s/it]

True label 1 Predicted label 0


In [148]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [149]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [150]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.20      0.67      0.30        12
           1       0.79      0.31      0.45        48

    accuracy                           0.38        60
   macro avg       0.49      0.49      0.37        60
weighted avg       0.67      0.38      0.42        60



## 4_1

In [151]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '4_1')

  2%|▏         | 1/60 [00:00<00:36,  1.60it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:42,  1.37it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:02<00:38,  1.48it/s]

...

True label 0 Predicted label 1


 98%|█████████▊| 59/60 [00:38<00:00,  1.58it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [00:39<00:00,  1.52it/s]

True label 1 Predicted label 1


In [152]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [153]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [154]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.21      0.67      0.32        12
           1       0.82      0.38      0.51        48

    accuracy                           0.43        60
   macro avg       0.51      0.52      0.42        60
weighted avg       0.70      0.43      0.48        60



## 3_1

In [155]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_1')

  2%|▏         | 1/60 [00:00<00:31,  1.90it/s]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:01<00:30,  1.88it/s]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:01<00:30,  1.89it/s]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [00:37<00:00,  1.39it/s]

True label 0 Predicted label 0


100%|██████████| 60/60 [00:38<00:00,  1.56it/s]

True label 1 Predicted label 0


In [156]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [157]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [158]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.21      1.00      0.34        12
           1       1.00      0.04      0.08        48

    accuracy                           0.23        60
   macro avg       0.60      0.52      0.21        60
weighted avg       0.84      0.23      0.13        60



## 3_2

In [170]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)
predictions = make_predictions(df_test, labels, '3_2')

  2%|▏         | 1/60 [00:01<01:11,  1.22s/it]

True label 1 Predicted label 0


  3%|▎         | 2/60 [00:02<01:13,  1.26s/it]

True label 1 Predicted label 0


  5%|▌         | 3/60 [00:03<01:05,  1.14s/it]

...

True label 0 Predicted label 0


 98%|█████████▊| 59/60 [01:23<00:01,  1.09s/it]

True label 0 Predicted label 1


100%|██████████| 60/60 [01:24<00:00,  1.41s/it]

True label 1 Predicted label 0


In [171]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [172]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [173]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.18      0.75      0.30        12
           1       0.73      0.17      0.27        48

    accuracy                           0.28        60
   macro avg       0.46      0.46      0.28        60
weighted avg       0.62      0.28      0.28        60

